In [1]:
from binary_models.load_se_rt import load_community_model, model_folder

In [2]:
import pandas as pd
import numpy as np
from micom.workflows import  grow, tradeoff
from micom.viz import plot_tradeoff, plot_growth

In [3]:
from binary_models.load_se_rt import load_community_model, model_folder
# first time running, add prep_files=True
# or if there are any model updates, or if you aren't sure but want to be
community, manifest = load_community_model(prep_files=False)

Set parameter Username
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "leghorn"
Read LP format model from file C:\Users\pino216\AppData\Local\Temp\tmpsqqm3rla.lp
Reading time = 0.04 seconds
: 3273 rows, 7403 columns, 29233 nonzeros


In [4]:
# Set variable to become new medium
grow_medium_to_change = community.medium
#Add or subtract reactions
#grow_medium_to_change["EX_glc__D_m"] = 0
grow_medium_to_change["EX_sucr_m"] = 1
#grow_medium_to_change["EX_leu__L_m"] = 1

In [5]:
grow_medium = pd.Series(grow_medium_to_change).to_frame('flux').reset_index()
grow_medium = grow_medium.rename(columns={'index':'reaction'})
grow_medium

,reaction,flux
0,EX_co2_m,1.99
1,EX_h_m,999999.00
2,EX_h2o_m,999999.00
3,EX_nh4_m,999999.00
4,EX_o2_m,999999.00
...,...,...
149,EX_photon630_m,1000.00
150,EX_photon650_m,1000.00
151,EX_photon670_m,1000.00
152,EX_photon690_m,1000.00


In [6]:
result_grow = grow(
    manifest,
    model_folder=model_folder,
    medium=grow_medium,
    tradeoff=0.01,
    threads=2,
    presolve=True
)

Output()

In [7]:
result_grow.growth_rates

,abundance,growth_rate,reactions,metabolites,taxon,tradeoff,sample_id
compartments,,,,,,,
Rhodosporidium,0.5,0.659540,2423,2076,Rhodosporidium,0.01,One
Synechococcus,0.5,0.010499,968,884,Synechococcus,0.01,One


In [8]:
result_grow.exchanges

,taxon,sample_id,tolerance,reaction,flux,abundance,metabolite,direction
0,Rhodosporidium,One,0.000001,EX_co2_e,20.057582,0.5,co2_e,export
3,Rhodosporidium,One,0.000001,EX_inost_e,0.008959,0.5,inost_e,export
16,Rhodosporidium,One,0.000001,EX_cu2_e,-0.000346,0.5,cu2_e,import
20,Rhodosporidium,One,0.000001,EX_ac_e,9.535148,0.5,ac_e,export
28,Rhodosporidium,One,0.000001,EX_h2o_e,20.212794,0.5,h2o_e,export
...,...,...,...,...,...,...,...,...
711,medium,One,0.000001,EX_h_m,3.757742,NaN,h_m,export
716,medium,One,0.000001,EX_cmp_m,-0.033906,NaN,cmp_m,import
727,medium,One,0.000001,EX_h2o_m,10.199468,NaN,h2o_m,export
733,medium,One,0.000001,EX_co2_m,10.023828,NaN,co2_m,export


In [9]:
exchanges = result_grow.exchanges.copy()

In [10]:
exchanges = exchanges[['taxon', 'reaction', 'flux', 'metabolite', 'direction']]

In [11]:
exchanges.to_csv("se_rt_exchanges.csv", index=False)

In [12]:
grow_plot = plot_growth(result_grow)

In [13]:
grow_plot.view()

In [ ]:
trades = tradeoff(
    manifest,
    model_folder,
    grow_medium,
    tradeoffs=np.arange(0.1, 1.0 + 1e-6, 0.1),
    threads=8,
    atol=None,
    rtol=None,
    presolve=False,
)

Output()

In [ ]:
tr_plot = plot_tradeoff(trades)

In [ ]:
tr_plot.view()